# convect each results into excel

In [2]:
import pandas as pd

df = pd.read_csv('results/summarized_results.csv')

# Sort the dataframe

df_sorted = df.drop(columns=['nn_model', 'number_of_clients'])
# Sort the dataframe by skew, dataset, and heterogeneity_class

# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
# Add a newline before each occurrence of 'fedavg' in the 'exp_type' column
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
# Sort the dataframe by the defined order
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('results.xlsx', index=False)

In [5]:
df=pd.read_csv('granular_results/noqs.csv')
# Define the order and bold the 'cornflqs' exp_type
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('noqs.xlsx', index=False)

In [7]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/qs1.csv')
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs1.xlsx', index=False)


In [6]:
df=pd.read_csv('granular_results/qs2.csv')
# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted = df
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs2.xlsx', index=False)

# calculate ranking 

In [3]:
def algo_ranking(dataset=None, skew=None, heterogeneity_class=None):
    import pandas as pd
    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv('results/summarized_results.csv')
    df.fillna('None', inplace=True)
    df = df[df['exp_type'] != 'oracle-centralized']
    df['exp_type'] = df['exp_type'] + '_' + df['params']
    
    # Filter the dataframe based on the input parameters
    if dataset:
        df = df[df['dataset'] == dataset]
    if skew:
        if skew == 'None':
            df = df[df['skew'] == skew]
        elif skew == 'quantity-skew':
            df = df[df['skew'] != 'None']
        else:
            df = df[df['skew'] == skew]
    if heterogeneity_class:
        df = df[df['heterogeneity_class'] == heterogeneity_class]

    # Step 2: Clean the data
    # Remove the '\pm' and uncertainty values from the 'accuracy' column
    df['accuracy'] = df['accuracy'].str.split(' ').str[0].astype(float)
    # Replace 'n/a' with NaN for missing values

    # Convert metric columns to numeric
    metrics = ['accuracy', 'ARI', 'AMI', 'hom', 'cmplt', 'vm']
    df[metrics] = df[metrics].apply(pd.to_numeric, errors='coerce')

    # Concatenate 'dataset', 'heterogeneity_class', and 'skew' into one column named 'group'
    df['group'] = df['dataset'] + '_' + df['heterogeneity_class'] + '_' + df['skew']
    # Drop the original 'dataset', 'heterogeneity_class', and 'skew' columns
    df = df.drop(columns=['dataset', 'heterogeneity_class', 'skew'])

    # Step 3: Group the data by the new 'group' column
    grouped = df.groupby('group')

    # Step 4: Rank algorithms within each group for each metric
    def rank_algorithms(group):
        for metric in metrics:
            group[f'{metric}_rank'] = group[metric].rank(ascending=False, method='min')
        return group

    ranked_groups = grouped.apply(rank_algorithms)
    best = ranked_groups.groupby('exp_type')[[f'{metric}_rank' for metric in metrics]].mean()
    best = best.sort_values(by='accuracy_rank')

    # Calculate the average accuracy for each exp_type
    avg_accuracy = ranked_groups.groupby('exp_type')['accuracy'].mean()
    best['avg_accuracy'] = avg_accuracy

    return best


In [5]:
algo_ranking()

/tmp/ipykernel_5669/4263264576.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_5669/4263264576.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
ifca_multiple,2.000000,1.076923,1.076923,1.076923,1.192308,1.076923,79.466154
cornflqs_euclidean,2.259259,1.629630,1.629630,1.555556,1.814815,1.629630,77.935926
hcfl_euclidean,3.111111,1.888889,1.851852,1.851852,1.888889,1.851852,75.638519
fedgroup_edc,4.481481,3.185185,3.222222,3.222222,3.444444,3.222222,74.636296
srfca_none,4.555556,2.074074,2.074074,2.111111,1.037037,2.074074,74.311852
fedprox_0.1,5.307692,NaN,NaN,NaN,NaN,NaN,64.805769
fedavg_None,6.000000,NaN,NaN,NaN,NaN,NaN,63.183704


In [6]:
algo_ranking(skew='quantity-skew-type-1')

/tmp/ipykernel_5669/4263264576.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_5669/4263264576.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
ifca_multiple,2.000000,1.000000,1.000000,1.000000,1.111111,1.000000,79.282222
cornflqs_euclidean,2.111111,1.666667,1.666667,1.555556,1.666667,1.666667,78.043333
srfca_none,3.444444,1.222222,1.222222,1.333333,1.111111,1.222222,76.491111
hcfl_euclidean,3.777778,3.555556,3.444444,3.444444,3.444444,3.444444,70.041111
fedgroup_edc,5.000000,4.444444,4.555556,4.555556,4.555556,4.555556,72.323333
fedprox_0.1,5.222222,NaN,NaN,NaN,NaN,NaN,61.845556
fedavg_None,6.333333,NaN,NaN,NaN,NaN,NaN,59.340000


In [7]:
algo_ranking(skew='quantity-skew-type-2')

/tmp/ipykernel_5669/4263264576.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_5669/4263264576.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
ifca_multiple,2.000000,1.250000,1.250000,1.250000,1.500000,1.250000,79.780000
hcfl_euclidean,2.555556,1.000000,1.000000,1.000000,1.000000,1.000000,77.872222
cornflqs_euclidean,2.888889,2.000000,2.000000,2.000000,2.444444,2.000000,76.616667
fedgroup_edc,4.333333,3.111111,3.111111,3.111111,3.555556,3.111111,73.672222
srfca_none,4.888889,2.666667,2.666667,2.666667,1.000000,2.666667,73.041111
fedprox_0.1,5.111111,NaN,NaN,NaN,NaN,NaN,69.916667
fedavg_None,5.666667,NaN,NaN,NaN,NaN,NaN,68.342222


In [8]:
algo_ranking(skew='quantity-skew')

/tmp/ipykernel_5669/4263264576.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_5669/4263264576.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
ifca_multiple,2.000000,1.117647,1.117647,1.117647,1.294118,1.117647,79.516471
cornflqs_euclidean,2.500000,1.833333,1.833333,1.777778,2.055556,1.833333,77.330000
hcfl_euclidean,3.166667,2.277778,2.222222,2.222222,2.222222,2.222222,73.956667
srfca_none,4.166667,1.944444,1.944444,2.000000,1.055556,1.944444,74.766111
fedgroup_edc,4.666667,3.777778,3.833333,3.833333,4.055556,3.833333,72.997778
fedprox_0.1,5.166667,NaN,NaN,NaN,NaN,NaN,65.881111
fedavg_None,6.000000,NaN,NaN,NaN,NaN,NaN,63.841111


In [9]:
algo_ranking(skew='None')

/tmp/ipykernel_5669/4263264576.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_5669/4263264576.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
cornflqs_euclidean,1.777778,1.222222,1.222222,1.111111,1.333333,1.222222,79.147778
ifca_multiple,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,79.371111
hcfl_euclidean,3.000000,1.111111,1.111111,1.111111,1.222222,1.111111,79.002222
fedgroup_edc,4.111111,2.000000,2.000000,2.000000,2.222222,2.000000,77.913333
srfca_none,5.333333,2.333333,2.333333,2.333333,1.000000,2.333333,73.403333
fedprox_0.1,5.625000,NaN,NaN,NaN,NaN,NaN,62.386250
fedavg_None,6.000000,NaN,NaN,NaN,NaN,NaN,61.868889


# EXP CSV generation

In [ ]:
import pandas as pd
import itertools

# Define the list of tuples for exp_type and params
exp_params_list = [
    ('fedavg', 'None'),
    ('fedprox', 0.1),
    ('cfl', 'None'),
    ('hcfl', 'euclidean'),
    ('fedgroup', 'madc'),
    ('fedgroup', 'edc'),
    ('ifca', 'best'),
    ('srfca', 'None'),
    ('cornflqs', '(1,10,10)'),
    ('cornflqs', '(10,1,10)'),
    ('cornflqs', '(1,1,20)'),
    ('oracle-centralized', 'None')
]

# Define the base data
base_data = [
    ['mnist', 'concept-shift-on-features', 'None'],
    ['mnist', 'concept-shift-on-labels', 'None'],
    ['mnist', 'features-distribution-skew', 'None'],
    ['fashion-mnist', 'concept-shift-on-features', 'None'],
    ['fashion-mnist', 'concept-shift-on-labels', 'None'],
    ['fashion-mnist', 'features-distribution-skew', 'None'],
    ['kmnist', 'concept-shift-on-features', 'None'],
    ['kmnist', 'concept-shift-on-labels', 'None'],
    ['kmnist', 'features-distribution-skew', 'None'],
    ['mnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['mnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['mnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['fashion-mnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['fashion-mnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['fashion-mnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['kmnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['kmnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['kmnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['fashion-mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['kmnist', 'features-distribution-skew', 'quantity-skew-type-2']
]

# Define fixed parameters
nn_model = 'linear'
number_of_clients = 100
num_samples_by_label = 50
num_clusters = 4
epochs = 10
rounds = 20
# Define the seed values
seed_values = [42, 43, 44, 45, 46]

# Create a list to store all the combinations
all_combinations = []

# Generate all combinations of base_data, exp_params_list, and seed_values
for base, exp_params, seed in itertools.product(base_data, exp_params_list, seed_values):
    combination = base + list(exp_params) + [seed, nn_model, number_of_clients, num_samples_by_label, num_clusters, epochs, rounds]
    all_combinations.append(combination)

# Convert the list of combinations to a DataFrame
columns = ['dataset', 'heterogeneity_class', 'skew', 'exp_type', 'params', 'seed', 'nn_model', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds']
df_combinations = pd.DataFrame(all_combinations, columns=columns)

# Reorder the columns as specified
df_combinations = df_combinations[['exp_type', 'params', 'dataset', 'nn_model', 'heterogeneity_class', 'skew', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds', 'seed']]

# Save the DataFrame to a CSV file
df_combinations.to_csv('exp_configs.csv', index=False)

In [4]:
import pandas as pd
import itertools

# Define the list of tuples for exp_type and params
exp_params_list = [
    ('fedavg', 'None'),
    ('fedprox', 0.1),
    ('cfl', 'None'),
    ('hcfl', 'euclidean'),
    ('fedgroup', 'edc'),
    ('ifca', 'best'),
    ('srfca', 'None'),
    ('cornflqs', '(1,10,10)'),
    ('cornflqs', '(10,1,10)'),
    ('cornflqs', '(1,1,20)'),
    ('oracle-centralized', 'None')
]

# Define the base data
base_data = [
    ['mnist', 'concept-shift-on-features', 'None'],
    ['mnist', 'concept-shift-on-labels', 'None'],
    ['mnist', 'features-distribution-skew', 'None'],
    ['fashion-mnist', 'concept-shift-on-features', 'None'],
    ['fashion-mnist', 'concept-shift-on-labels', 'None'],
    ['fashion-mnist', 'features-distribution-skew', 'None'],
    ['kmnist', 'concept-shift-on-features', 'None'],
    ['kmnist', 'concept-shift-on-labels', 'None'],
    ['kmnist', 'features-distribution-skew', 'None'],
    ['mnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['mnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['mnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['fashion-mnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['fashion-mnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['fashion-mnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['kmnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['kmnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['kmnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['fashion-mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['kmnist', 'features-distribution-skew', 'quantity-skew-type-2']
]

# Define fixed parameters
nn_model = 'linear'
number_of_clients = 100
num_samples_by_label = 50
num_clusters = 4
default_epochs = 10
rounds = 20
# Define the seed values
seed_values = [42, 43, 44, 45, 46]

# Create a list to store all the combinations
all_combinations = []

# Generate all combinations of base_data, exp_params_list, and seed_values
for base, (exp_type, params), seed in itertools.product(base_data, exp_params_list, seed_values):
    epochs = 100 if exp_type == 'oracle-centralized' else default_epochs
    combination = base + [exp_type, params, seed, nn_model, number_of_clients, num_samples_by_label, num_clusters, epochs, rounds]
    all_combinations.append(combination)

# Convert the list of combinations to a DataFrame
columns = ['dataset', 'heterogeneity_class', 'skew', 'exp_type', 'params', 'seed', 'nn_model', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds']
df_combinations = pd.DataFrame(all_combinations, columns=columns)

# Reorder the columns as specified
df_combinations = df_combinations[['exp_type', 'params', 'dataset', 'nn_model', 'heterogeneity_class', 'skew', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds', 'seed']]

# Save the DataFrame to a CSV file
df_combinations.to_csv('exp_configs.csv', index=False)
